In [7]:
// SETUP
use std::fs::File;
use std::collections::HashMap;
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

In [65]:
// DAY01

const TOTAL: i64 = 2020;

fn problem01() {    
    let nums: Vec<i64> = read_integers(&data_file(1)).unwrap();    
    let mut reg = HashMap::new();

    for (i, n) in nums.iter().enumerate() {
        let rest = TOTAL - n;
        if reg.contains_key(&rest) {
            println!("{} * {} = {}", n, rest, n * rest);
        }
        reg.insert(n, i);
    }
    
    for (i, n1) in nums.iter().enumerate() {
        for j in (i + 1)..nums.len() {
            let n2 = nums[j];
            let rest = TOTAL - n1 - n2;
            match reg.get(&rest) {
                Some(k) if k > &j => {
                    println!("{} * {} * {} = {}", n1, n2, rest, n1 * n2 * rest)
                },
                _ => (),
            }
        }
    }
}

problem01();

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [50]:
// DAY02
#[derive(Debug, PartialEq)]
struct PasswordDesc {
    cnt1: usize,
    cnt2: usize,
    ch: char,
    pwd: String,
}

impl FromStr for PasswordDesc {
    type Err = std::num::ParseIntError;
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let parts: Vec<&str> = s.split(' ').collect();
        let counts: Vec<usize> = parts[0]
            .split('-')
            .map(|p| p.parse().unwrap())
            .collect();
        Ok(PasswordDesc { 
            cnt1: counts[0], 
            cnt2: counts[1], 
            ch: parts[1].as_bytes()[0] as char, 
            pwd: parts[2].to_owned()})
    }
}

fn is_valid1(p: &PasswordDesc) -> bool {
    let cnt = p.pwd.matches(p.ch).count();
    p.cnt1 <= cnt && cnt <= p.cnt2
}

fn is_valid2(p: &PasswordDesc) -> bool {
    let pwd = p.pwd.as_bytes();
    let ch = p.ch as u8;
    (pwd[p.cnt1 - 1] == ch) ^ (pwd[p.cnt2 - 1] == ch)
}

fn problem02() {
    let passwords: Vec<PasswordDesc> = 
        read_lines(&data_file(2)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();

    println!("Number of valid passwords: {}, {}", 
        passwords.iter().filter(|x| is_valid1(x)).count(), 
        passwords.iter().filter(|x| is_valid2(x)).count());
}

problem02();

Number of valid passwords: 548, 502


In [69]:
const TREE: u8 = '#' as u8;

fn count_trees(rows: &Vec<String>, dx: usize, dy: usize) -> usize {
    let h = rows.len();
    if h == 0 {
        return 0;
    }
    let w = rows[0].len();
    let mut x: usize = 0;
    let mut res: usize = 0;
    for y in (0..h).step_by(dy) {
        res += (rows[y].as_bytes()[x % w] == TREE) as usize;
        x += dx;
    }
    res
}

fn problem03() {
    let rows = read_lines(&data_file(3)).unwrap();
    let res1 = count_trees(&rows, 3, 1);
    println!("Num trees 1: {}", res1);
    
    let res2: usize = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
        .iter()
        .map(|(dx, dy)| count_trees(&rows, *dx, *dy))
        .product();
    println!("Num trees 2: {}", res2);
}

problem03();

Num trees 1: 242
Num trees 2: 2265549792
